## Support Vector Machines

In [ ]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "sk2torch"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
import json
import numpy as np
from sklearn.svm import SVC
from hummingbird.ml import convert
import torch
import ezkl
import os

# Create a dataset of two Gaussians. There will be some overlap
# between the two classes, which adds some uncertainty to the model.
xs = np.concatenate(
    [
        np.random.random(size=(256, 2)) + [1, 0],
        np.random.random(size=(256, 2)) + [-1, 0],
    ],
    axis=0,
)
ys = np.array([False] * 256 + [True] * 256)

# Train an SVM on the data and wrap it in PyTorch.
sk_model = SVC(probability=True)
sk_model.fit(xs, ys)
model = convert(sk_model, "torch").model

model

Executor(
  (_operators): ModuleList(
    (0): SVC()
  )
)

In [ ]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [ ]:
spaced = np.linspace(-2, 2, num=25)
grid_xs = torch.tensor([[x, y] for x in spaced for y in spaced], requires_grad=True)
# export to onnx format
# !!!!!!!!!!!!!!!!! This will flash a warning but it is fine !!!!!!!!!!!!!!!!!!!!!

# Input to the model
shape = xs.shape[1:]
x = grid_xs[0:1]
# Export the model
torch.onnx.export(model,               # model being run
                  # model input (or a tuple for multiple inputs)
                  x,
                  # where to save the model (can be a file or file-like object)
                  "network.onnx",
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names=['input'],   # the model's input names
                  output_names=['output'],  # the model's output names
                  dynamic_axes={'input': {0: 'batch_size'},    # variable length axes
                                'output': {0: 'batch_size'}})

d = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data=[d])

# Serialize data into file:
json.dump(data, open("input.json", 'w'))

In [ ]:
!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path)
assert res == True

In [ ]:
cal_path = os.path.join("calibration.json")

data_array = ((grid_xs[0:20]).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

# Serialize data into file:
json.dump(data, open(cal_path, 'w'))

await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources")

True

In [ ]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [ ]:
# srs path
res = await ezkl.get_srs( settings_path)

In [ ]:
# now generate the witness file 

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [ ]:
# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK

res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [ ]:
# GENERATE A PROOF

proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        
        "single",
    )

print(res)
assert os.path.isfile(proof_path)

In [ ]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        
    )

assert res == True
print("verified")

# Linear SVC

In [ ]:
### Linear SVC

# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "sk2torch"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
import json
import numpy as np
from sklearn.svm import LinearSVC
import sk2torch
import torch
import ezkl
import os

xs = np.concatenate(
    [
        np.random.random(size=(256, 2)) + [1, 0],
        np.random.random(size=(256, 2)) + [-1, 0],
    ],
    axis=0,
)
ys = np.array([False] * 256 + [True] * 256)

# Train an SVM on the data and wrap it in PyTorch.
sk_model = LinearSVC(dual='auto')
sk_model.fit(xs, ys)
model = sk2torch.wrap(sk_model)

In [ ]:
# export to onnx format
# !!!!!!!!!!!!!!!!! This will flash a warning but it is fine !!!!!!!!!!!!!!!!!!!!!

# Input to the model
shape = xs.shape[1:]
x = grid_xs[0:1]
torch_out = model.predict(x)
# Export the model
torch.onnx.export(model,               # model being run
                  # model input (or a tuple for multiple inputs)
                  x,
                  # where to save the model (can be a file or file-like object)
                  "network.onnx",
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names=['input'],   # the model's input names
                  output_names=['output'],  # the model's output names
                  dynamic_axes={'input': {0: 'batch_size'},    # variable length axes
                                'output': {0: 'batch_size'}})

d = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_shapes=[shape],
            input_data=[d],
            output_data=[o.reshape([-1]).tolist() for o in torch_out])

# Serialize data into file:
json.dump(data, open("input.json", 'w'))

In [ ]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [ ]:
!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path)
assert res == True

res = await ezkl.calibrate_settings(data_path, model_path, settings_path, "resources")
assert res == True